# A showcase of the different functionalities of the framework

## Workflow

- **Step 1: Gather the Test Inputs**
  - The test session with all the pictures and coordinates
  - The global coordinates of the device
- **Step 2: check for relevant Reference data**
  - use the global coordinates to find all the reference data that is geo-referenced close enough (GPS precision)
- **Step 3: 2D Check**
  - Compare all the test images against all the reference images
  - Find which session has the highest match rate
  - Find which Image has the highest match rate
  - Calculate the transformation between the two images
  - calculate the inverse transformation to give the test data a Reference global position
- **Step 4: 3D Check**
  - Compare the test mesh against relevant point clouds
  - Compare the test mesh against BIM models
  - Perform a CCP for the final alignment
- **Step 5: Choosing final Position**
  - Use the different results from all the methods to come to a best position
  - Send the Position and rotation back to the device

## Session Functionality
A session contains all the data from the folder: the images, meshes and their locations. It also contains the geo-reference position and rotation

### Importing and Selecting Sessions
Sessions can either be directly imported from a path, or be selected from a parent directory based on the distance to the reference point.

In [ ]:
import numpy as np
import session

# you can import all the close enough sessions from a parent directory 
sessionsFolderLocation = "/Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-11 Testbuilding Campus/RAW Data"
referencePoint = np.array([0,0,0])
maxDistance = 10
sessions = session.find_close_sessions(sessionsFolderLocation, referencePoint, maxDistance)

In [ ]:
# you can also import a single session from the directory
sessionDirectory = "/Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-11 Testbuilding Campus/RAW Data/Hololens/session-2021-11-25 16-09-47"
singleSession = session.Session().from_path(sessionDirectory)
print (singleSession.__dict__)

### Bounding Area
Each session has a bounding area to determine if the reference position is close enough to the data

In [ ]:
boundingBox = singleSession.get_bounding_box()
boundingRadius = singleSession.get_bounding_radius()

print("The axis aligned bounding box (2x3 array) min and max corner points ", boundingBox)
print ("The bounding radius from the reference Center:", boundingRadius)

### Image transforms
The image transform contains the Image file, and it's transform in session space

In [ ]:
# an image transform is stored in 
image1 = singleSession.imageTransforms[0]
print(image1.__dict__)

#### The Image
The image is stored as an openCV color image.

In [ ]:
from matplotlib import pyplot as plt
import cv2

cv2Image = image1.get_cv2_image()

plt.imshow(cv2.cvtColor(cv2Image, cv2.COLOR_BGR2RGB))
plt.title('cv2 Color Image')
plt.show()

### The Geometries

In [ ]:
import positioning3D as pos3D
import session

sessionDirectory = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2021-11-25 16-17-19"
singleSession = session.Session().from_path(sessionDirectory)
#print(singleSession.geometries)
pos3D.show_geometries(singleSession.geometries, True)

## Positioning 2D Functionality
All the different methods and functions to calculate the relative position of a Session using 2D data

### Comparing Images
2 images can be compared against each other to find the corresponding matches and quality of the match.

In [ ]:
#import 2 images from a session
sessionDirectory = "/Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-11 Testbuilding Campus/RAW Data/Hololens/session-2021-11-25 16-09-47"
singleSession = session.Session().from_path(sessionDirectory)
image1 = singleSession.imageTransforms[0]
image2 = singleSession.imageTransforms[1]
image3 = singleSession.imageTransforms[2]


#Find the matches between the 2 images
import compareImage as ci

matchScore, matches, keypoints1, keypoints2 = ci.find_matches(image1, image2)

### Comparing Sessions
You can also compare 2 sessions to each other, depending

In [ ]:
session2Directory = "/Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-11 Testbuilding Campus/RAW Data/Hololens/session-2021-11-25 16-17-19"
Session2 = session.Session().from_path(session2Directory)

import positioning2D as pos2D

bestResults = pos2D.compare_session(singleSession, Session2)

print(bestResults.__dict__)

### Calculating the Transformation matrix
The transformation between 2 matched images can be determined using the Essential matrix.

In [ ]:
import compareImage as ci

E, E_direct,F, pts1,pts2, imMatches = ci.calculate_transformation_matrix(image1, image2, matches, keypoints1, keypoints2)

### Triangulating the Camera Position
When the transformation matrix is calculated, the next step is to determine the camera pose.
Because the Essential matrix is correct up to a scale factor, that scale factor needs to be determined first.

#### Minimum distance between known points
The first method requires 2 reference images and 1 test image. Since the location of the reference image is known and the direction of the test image is also determined, The location of the test image can be calculated by finding the minimal distance between the 2 estimated positions.

In [ ]:
import transform

matchScore2, E2, imMatches2 = ci.compare_image(image1, image2)
matchScore3, E3, imMatches3 = ci.compare_image(image1, image3)

newPos, rot1, pos1, pos2, scale = transform.triangulate_session(image2, image3, E2, E3)

print("Estimated position:", newPos)
print("Actual position:", image1.pos)

#### Reference Session Scaling
Since the location of the reference images is known, we can calculate the feature scaling by matching 2 reference images. By then matching the test image to one of the matched images, the scale is already known.

In [ ]:
matchScore, E, imMatches = ci.compare_image(image2, image3)
scale = transform.get_session_scale(image2, image3, E)

matchScore, E, imMatches = ci.compare_image(image2, image1)
direction, rotation = transform.get_translation(E)

newPos = image2.pos + direction * scale

print("The session scale :", scale)
print("Estimated position:", newPos)
print("Actual position:", image1.pos)

#### Using the 3D scene
The session data might also contain 3D data, like a mesh or point cloud. They are also localized, so once the features are calculated for an image, a ray can be cast from a point to determine the global scale.

In [ ]:
import positioning3D



## Positioning 3D Functionality
Using mesh data to calculate the transformations, the pipeline uses open3D as the main framework for importing, displaying and matching geometry

### Importing meshes and point clouds


In [ ]:
import positioning3D as pos3D
import open3d as o3d

meshPath = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/mesh-2021-11-25 16-16-01.obj"
mesh = pos3D.import_mesh(meshPath)
pos3D.show_geometries([mesh])

In [ ]:
import positioning3D as pos3D

pcdPath = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/S003-SW-002.ply"
pcd = pos3D.import_point_cloud(pcdPath)

pos3D.show_geometries([pcd])

#### Converting and Downsampling
Meshes have to be converted to point clouds to extract features

In [ ]:
meshPath = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/positioning/images/ref/mesh-2021-11-25 16-16-01.obj"
mesh = pos3D.import_mesh(meshPath)
nrOfPoints = 10000
superSampleFactor = 2
pcd = pos3D.to_pcd(mesh, nrOfPoints, superSampleFactor)

### Aligning Point clouds
2 point clouds can be aligned using a FPFH 3D feature matching algorithm.

In [1]:
import positioning3D as pos3D
import copy

pcd1 = pos3D.import_point_cloud("/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/pcd1.pcd")
#pcd2 = pos3D.import_point_cloud("/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/pcd2.pcd")
pcd2 = pos3D.import_point_cloud("/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/S003-SW-002.ply")

transformation = pos3D.get_pcd_transformation(pcd1, pcd2, 0.1)
print("This is the estimated transformation: \n", transformation)
movedPcd = copy.deepcopy(pcd1)
movedPcd.transform(transformation)
pos3D.show_geometries([pcd1,movedPcd, pcd2])

Importing point cloud from: /Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/pcd1.pcd
Importing complete: PointCloud with 100000 points.
Importing point cloud from: /Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/S003-SW-002.ply
Importing complete: PointCloud with 7638931 points.
:: Compute FPFH feature with search radius 0.500.
:: Compute FPFH feature with search radius 0.500.
:: Apply fast global registration with distance threshold 0.500
This is the estimated transformation: 
 [[-1.95046263e-01 -1.45380299e-02 -9.80686291e-01  5.22049811e+00]
 [-9.80787476e-01 -7.68041484e-04  1.95077774e-01 -1.10544387e+00]
 [-3.58925425e-03  9.99894022e-01 -1.41089140e-02  1.96711010e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


## Global Alignment
Once a number of estimated transformations are determined, you can compare them to find the best one

## Full example
This a a workflow example going over all the steps

In [1]:
import numpy as np
import session

referencePoint = np.array([0,0,0])
maxDistance = 10

testSessionDir = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2021-11-25 16-17-19"
refSessionDir = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2021-11-25 16-09-47"
testSession = session.Session().from_path(testSessionDir)
referenceSession = session.Session().from_path(refSessionDir)
closeEnoughSessions = [referenceSession]




In [ ]:
#2D estimation
import positioning2D as pos2D
import transform

ImageTransformations = pos2D.get_2D_transformation(testSession, closeEnoughSessions)

for transformation in ImageTransformations:
    print("These are the estimated transformations:")
    E2 = transformation[0].transMatrix
    E3 = transformation[1].transMatrix
    image2 = transformation[0].refImage
    image3 = transformation[1].refImage
    newPos, rot1, pos1, pos2, scale = transform.triangulate_session(image2, image3, E2, E3)

    print("Estimated position:", newPos)
    print("Actual position:", transformation[0].testImage.pos)

In [2]:
#3D estimation
import positioning3D as pos3D

MeshTransformation = pos3D.get_3D_transformation(testSession, closeEnoughSessions, 0.1)

Starting Comparing: 1 Against 5 Geometries
converting mesh to PCD with 100000 points
Converting complete PointCloud with 100000 points.
converting mesh to PCD with 100000 points
Converting complete PointCloud with 100000 points.
:: Compute FPFH feature with search radius 0.500.
:: Compute FPFH feature with search radius 0.500.
:: Apply fast global registration with distance threshold 0.500
converting mesh to PCD with 100000 points
Converting complete PointCloud with 100000 points.
:: Compute FPFH feature with search radius 0.500.
:: Compute FPFH feature with search radius 0.500.
:: Apply fast global registration with distance threshold 0.500
converting mesh to PCD with 100000 points
Converting complete PointCloud with 100000 points.
:: Compute FPFH feature with search radius 0.500.
:: Compute FPFH feature with search radius 0.500.
:: Apply fast global registration with distance threshold 0.500
converting mesh to PCD with 100000 points
Converting complete PointCloud with 100000 points.


In [3]:

import copy

for transformations in MeshTransformation:
    print("here are the estimated transformations:")
    for i, element in enumerate(transformations):
        print(element)
        transMesh = copy.deepcopy(testSession.geometries[0])
        transMesh.transform(element)
        pos3D.show_geometries([transMesh, closeEnoughSessions[0].geometries[i]])

here are the estimated transformations:
[[ 0.69613836  0.38419272 -0.60645473 -7.32581588]
 [-0.71597569  0.30960796 -0.62571697 -1.09323937]
 [-0.05263269  0.86979243  0.49060262  5.41529037]
 [-0.         -0.         -0.          1.        ]]
[[  0.73699287  -0.54304891   0.40241693 -11.68871357]
 [  0.62595815   0.77298948  -0.10326503  -1.96709205]
 [ -0.25498609   0.32800175   0.90961362   7.88620969]
 [ -0.          -0.          -0.           1.        ]]
[[ 0.69711454  0.01118136  0.71687258 -9.14406207]
 [ 0.0225307   0.99904289 -0.0374922  -0.14724744]
 [-0.71660567  0.042288    0.69619541  9.92986617]
 [-0.         -0.         -0.          1.        ]]
[[ -0.36248391  -0.17178131  -0.91602216 -10.31781429]
 [  0.02834894  -0.98444437   0.17339439  -3.76785748]
 [ -0.93155877   0.03688442   0.36171507   4.23576794]
 [  0.           0.          -0.           1.        ]]
[[-0.86667598 -0.49313231  0.07545375 -2.00919593]
 [ 0.49654465 -0.83811859  0.22583321 -5.44931032]
 [-0.0

In [ ]:
transMesh = copy.deepcopy(testSession.geometries[0]).transform(element)
pos3D.show_geometries([transMesh, closeEnoughSessions[0].geometries[0]])

In [2]:
import positioning3D as pos3D
import numpy as np
import copy

meshPath = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2021-11-25 16-17-19/mesh-2021-11-25 16-17-19.obj"
mesh = pos3D.import_mesh(meshPath)
nrOfPoints = 100000
superSampleFactor = 1
pcdTest = pos3D.to_pcd(mesh, nrOfPoints, superSampleFactor)

meshPath = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2021-11-25 16-09-47/mesh-2021-11-25 16-16-01.obj"
mesh = pos3D.import_mesh(meshPath)
nrOfPoints = 100000
superSampleFactor = 1
pcdRef = pos3D.to_pcd(mesh, nrOfPoints, superSampleFactor)

#pcdRef = pos3D.import_point_cloud("/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/S003-SW-002.ply")

transformation = pos3D.get_pcd_transformation(pcdTest, pcdRef, 0.1)
print("This is the estimated transformation: \n", transformation)
movedPcd = copy.deepcopy(pcdTest)
movedPcd.transform(transformation)
pos3D.show_geometries([pcdTest,movedPcd, pcdRef],True)

Importing mesh from: /Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2021-11-25 16-17-19/mesh-2021-11-25 16-17-19.obj
Importing complete: TriangleMesh with 63197 points and 21839 triangles.
converting mesh to PCD with 100000 points
Converting complete PointCloud with 100000 points.
Importing mesh from: /Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2021-11-25 16-09-47/mesh-2021-11-25 16-16-01.obj
Importing complete: TriangleMesh with 525325 points and 183107 triangles.
converting mesh to PCD with 100000 points
Converting complete PointCloud with 100000 points.
:: Compute FPFH feature with search radius 0.500.
:: Compute FPFH feature with search radius 0.500.
:: Apply fast global registration with distance threshold 0.500
This is the estimated transformation: 
 [[ 9.99998793e-01  1.19977509e-03  9.86784993e-04  4.98610276e-01]
 [-1.20052025e-03  9.99998994e-01  7.54893234e-04 -2.69928310e-